<a href="https://colab.research.google.com/github/annabelle1217/computer-vision-mediapipe/blob/main/colab/body_language_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Body Language Decoder

## Table of Content
0. [Install and Import Dependencies](#install)
1. [Detection using MediaPipe](#detection) 
2. [Feature Extraction](#feature-extraction) 
    1. [Write Columns Head in CSV File](#csv-header)
    2. [Extract Features of Assigned Class](#save-coordinates)
3. [Train Custom Model Using Scikit Learn](#model)
    1. [Load and Preprocess Input Data](#load-input)
    2. [Train Machine Learning Classification Models](#training)
    3. [Evaluate and Serialize Model](#evaluate)
4. [Real-time Detections with Model](#real-time-detection)

## 0. Install and Import Dependencies <a id="install"></a>

In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

     |████████████████████████████████| 32.2 MB 44 kB/s 


In [ ]:
from mediapipe import solutions as mp
import cv2
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from IPython.display import display, Javascript, Image
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
from PIL import Image
import base64
import html
import io
import pickle
import os

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

# Helper functions for video frame extraction and drawing landmarks
def js_to_image(js_reply):
  """
  Params:js_reply: JavaScript object containing image from webcam
  Returns:img: OpenCV BGR image
  """
  # Decode base64 image
  image_bytes = base64.b64decode(js_reply.split(',')[1])
  # Convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # Decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

def drawing_array_to_bytes(drawing_array):
  """
  input: drawing_array: landmarks from holistic result
  output: drawing_bytes: string, encoded from drawing_array
  """
  drawing_PIL = Image.fromarray(drawing_array, 'RGB')
  iobuf = io.BytesIO()
  drawing_PIL.save(iobuf, format='png')
  drawing_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))
  return drawing_bytes
  
def video_frame(label, bytes):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bytes))
  return data

## 1. Detection using MediaPipe <a id="detection"></a>

In [ ]:
# Start streaming video from webcam
video_stream()
# Label for video
label_html = 'Holistic Model Detection'

drawing_lm = ''

# Initiate holistic model
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

  while True:
      js_reply = video_frame(label_html, drawing_lm)
      
      if not js_reply:
          break
        
      # Convert JS response to OpenCV Image
      image = js_to_image(js_reply["img"])

      # Recolor feed
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      # Make detections
      results = holistic.process(image)

      if results.pose_landmarks and results.face_landmarks:
        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS)

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS)
            
      # Show each frame
      # cv2_imshow(image)

      # Convert overlay of landmarks into bytes
      lm_bytes = drawing_array_to_bytes(image)
      
      # Update landmarks so next frame gets new overlay
      drawing_lm = lm_bytes

<IPython.core.display.Javascript object>

In [ ]:
results.pose_landmarks

landmark {
  x: 0.6370499134063721
  y: 0.7511494159698486
  z: -1.7452490329742432
  visibility: 0.9938176274299622
}
landmark {
  x: 0.6679985523223877
  y: 0.6638563871383667
  z: -1.6864700317382812
  visibility: 0.9958872199058533
}
landmark {
  x: 0.691382646560669
  y: 0.6649967432022095
  z: -1.686693549156189
  visibility: 0.994044840335846
}
landmark {
  x: 0.7168596386909485
  y: 0.6661290526390076
  z: -1.6876106262207031
  visibility: 0.9957603812217712
}
landmark {
  x: 0.5916308164596558
  y: 0.6590386033058167
  z: -1.7010301351547241
  visibility: 0.9963436126708984
}
landmark {
  x: 0.562659740447998
  y: 0.6584774255752563
  z: -1.7018249034881592
  visibility: 0.9941062331199646
}
landmark {
  x: 0.53446364402771
  y: 0.6598090529441833
  z: -1.7029060125350952
  visibility: 0.9953691959381104
}
landmark {
  x: 0.7326816320419312
  y: 0.698592483997345
  z: -1.121795415878296
  visibility: 0.9955944418907166
}
landmark {
  x: 0.48172909021377563
  y: 0.7019378542900

## 2. Feature Extraction <a id="feature-extraction"></a>

### Write Columns Head in CSV File <a id="csv-header"></a>

In [ ]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
if not os.path.exists("data"):
    os.mkdir("data")

with open("data/body_language_coords.csv", mode="w", newline="" ) as f:
    csv_writer = csv.writer(f, delimiter=",", quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Extract Features of Assigned Class <a id="save-coordinates"></a>

- Repeat this step to save features of different targets.
- Make sure your samples cover different scenario of the target.

In [ ]:
class_name = "right cross"

In [ ]:
# Start streaming video from webcam
video_stream()
# Label for video
label_html = "Feature Extraction - Change 'class_name' and Repeat This Step"

drawing_lm = ''

with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

  while True:
      js_reply = video_frame(label_html, drawing_lm)
      
      if not js_reply:
          break
        
      # Convert JS response to OpenCV Image
      image = js_to_image(js_reply["img"])
            
      # Recolor feed
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      # Make detections
      results = holistic.process(image)

      # Draw face landmarks
      mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS)

      # Right hand
      mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

      # Left hand
      mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

      # Pose 
      mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS)
      
      # Export coordinates
      if results.pose_landmarks and results.face_landmarks:
          # Extract pose landmarks
          pose = results.pose_landmarks.landmark
          pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                    for landmark in pose]).flatten())

              
          # Extract face landmarks
          face = results.face_landmarks.landmark
          face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                    for landmark in face]).flatten())
          
#             # Extract right hand landmarks
#             right_hand = results.right_hand_landmarks.landmark
#             right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
#                                             for landmark in right_hand]).flatten())
          
          # Concate row
          row = pose_row + face_row 
          
          # Append class name
          row.insert(0, class_name)
          
          # Export to CVS
          with open("data/body_language_coords.csv", mode="a", newline="" ) as f:
              csv_writer = csv.writer(f, delimiter=",", quotechar='"',quoting=csv.QUOTE_MINIMAL)
              csv_writer.writerow(row)
        
      # Convert overlay of landmarks into bytes
      lm_bytes = drawing_array_to_bytes(image)
      
      # Update landmarks so next frame gets new overlay
      drawing_lm = lm_bytes

<IPython.core.display.Javascript object>

## 3. Train Custom Model Using Scikit Learn <a id="model"></a>

### Load and Preprocess Input Data <a id="load-input"></a>

In [ ]:
df = pd.read_csv("data/body_language_coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,z3,v3,x4,y4,z4,v4,x5,y5,z5,v5,x6,y6,z6,v6,x7,y7,z7,v7,x8,y8,z8,v8,x9,y9,z9,v9,x10,y10,z10,...,x492,y492,z492,v492,x493,y493,z493,v493,x494,y494,z494,v494,x495,y495,z495,v495,x496,y496,z496,v496,x497,y497,z497,v497,x498,y498,z498,v498,x499,y499,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,raise,0.631438,0.702926,-1.435961,0.999407,0.662848,0.603750,-1.414533,0.998656,0.689298,0.600185,-1.414138,0.999022,0.714726,0.597403,-1.415169,0.998684,0.591162,0.610053,-1.416325,0.998798,0.564376,0.610905,-1.417764,0.999077,0.538183,0.612110,-1.418415,0.998745,0.744254,0.621329,-1.016978,0.998867,0.497170,0.648514,-1.021327,0.998785,0.676355,0.783157,-1.263639,...,0.656519,0.740157,-0.055829,0.0,0.663962,0.733688,-0.054499,0.0,0.674547,0.738232,-0.021864,0.0,0.654300,0.742731,-0.059528,0.0,0.651839,0.744819,-0.042832,0.0,0.671101,0.601727,0.005078,0.0,0.663997,0.608149,-0.003937,0.0,0.660714,0.613538,-0.013599,0.0,0.736219,0.590027,0.019222,0.0,0.743514,0.579365,0.020064,0.0
1,raise,0.637114,0.636899,-1.794069,0.999430,0.671705,0.545511,-1.716301,0.998760,0.697153,0.545655,-1.716009,0.999081,0.721033,0.546463,-1.716916,0.998786,0.594301,0.547147,-1.732759,0.998887,0.566670,0.548984,-1.734188,0.999117,0.540522,0.551544,-1.734719,0.998823,0.757749,0.584494,-1.095068,0.998935,0.498135,0.593925,-1.178464,0.998876,0.683611,0.723060,-1.539304,...,0.673292,0.644125,-0.054181,0.0,0.680529,0.637955,-0.052513,0.0,0.690710,0.644583,-0.022978,0.0,0.671221,0.646217,-0.057745,0.0,0.668856,0.649048,-0.042568,0.0,0.686914,0.517637,0.011187,0.0,0.680070,0.522575,0.002491,0.0,0.676726,0.526563,-0.006540,0.0,0.747392,0.510261,0.025007,0.0,0.754534,0.497969,0.026257,0.0
2,raise,0.630237,0.618040,-1.777059,0.999439,0.667491,0.528129,-1.704341,0.998849,0.692607,0.529765,-1.704026,0.999119,0.715262,0.532658,-1.704888,0.998874,0.587735,0.527511,-1.719010,0.998961,0.559877,0.528476,-1.720420,0.999131,0.533213,0.530958,-1.720997,0.998886,0.750962,0.572085,-1.090727,0.998985,0.489582,0.569127,-1.168403,0.998943,0.676895,0.705074,-1.525105,...,0.638831,0.638169,-0.056360,0.0,0.646328,0.632164,-0.055159,0.0,0.658830,0.637474,-0.024476,0.0,0.636541,0.640324,-0.059980,0.0,0.634602,0.642847,-0.043986,0.0,0.656198,0.514356,0.007620,0.0,0.648779,0.519304,-0.001109,0.0,0.644939,0.523190,-0.010361,0.0,0.719711,0.505704,0.019313,0.0,0.726958,0.495049,0.020179,0.0
3,raise,0.630677,0.622328,-1.711261,0.999459,0.667934,0.528483,-1.648443,0.998936,0.692684,0.529701,-1.648004,0.999166,0.715258,0.531469,-1.648941,0.998960,0.587896,0.529560,-1.659198,0.999036,0.559929,0.530852,-1.660561,0.999164,0.533282,0.532930,-1.661199,0.998955,0.751553,0.567917,-1.085537,0.999042,0.489740,0.568975,-1.135746,0.999008,0.677128,0.705524,-1.472328,...,0.653464,0.659709,-0.054610,0.0,0.660638,0.653618,-0.053306,0.0,0.670919,0.659104,-0.023475,0.0,0.651395,0.661897,-0.058143,0.0,0.649059,0.664461,-0.042605,0.0,0.669105,0.533429,0.009110,0.0,0.662184,0.539028,0.000467,0.0,0.658704,0.543612,-0.008533,0.0,0.731311,0.520716,0.021528,0.0,0.737867,0.509952,0.022415,0.0
4,raise,0.634418,0.627401,-1.746777,0.999433,0.669279,0.531694,-1.676492,0.998987,0.693041,0.531430,-1.676123,0.999145,0.715915,0.531500,-1.677023,0.999002,0.588477,0.535310,-1.690455,0.999083,0.560168,0.537021,-1.691735,0.999152,0.533611,0.539365,-1.692499,0.999000,0.752122,0.563903,-1.077919,0.999023,0.489821,0.575505,-1.138849,0.999025,0.681306,0.707952,-1.496501,...,0.654748,0.666124,-0.055633,0.0,0.662082,0.660183,-0.054475,0.0,0.673263,0.664450,-0.023663,0.0,0.652470,0.668409,-0.059239,0.0,0.650334,0.670570,-0.043231,0.0,0.671273,0.536858,0.007358,0.0,0.663907,0.542616,-0.001371,0.0,0.660154,0.547379,-0.010668,0.0,0.733895,0.526150,0.019079,0.0,0.740722,0.515479,0.019839,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
X = df.drop("class", axis=1)
y = df["class"]

In [ ]:
y.value_counts()

raise          29
right cross    20
left cross     19
Name: class, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)

### Train Machine Learning Classification Models <a id="training"></a>

In [ ]:
pipelines = {
    "lr":make_pipeline(StandardScaler(), LogisticRegression()),
    "svc":make_pipeline(StandardScaler(), SVC()),
    "rf":make_pipeline(StandardScaler(), RandomForestClassifier()),
    "knn":make_pipeline(StandardScaler(), KNeighborsClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train.ravel())
    fit_models[algo] = model

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Evaluate and Serialize Model <a id="evaluate"></a>

In [ ]:
# Show accuracy
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, accuracy_score(y_test, pred))

lr 1.0
svc 0.9285714285714286
rf 1.0
knn 0.7857142857142857


In [ ]:
# Show confusion matrix
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, confusion_matrix(y_test, pred), sep="\n", end="\n\n")

lr
[[7 0 0]
 [0 2 0]
 [0 0 5]]

svc
[[7 0 0]
 [0 2 0]
 [1 0 4]]

rf
[[7 0 0]
 [0 2 0]
 [0 0 5]]

knn
[[5 0 2]
 [0 2 0]
 [1 0 4]]



In [ ]:
# Create folder if it does not exist
if not os.path.exists("generated_model"):
    os.mkdir("generated_model")
    
# Save model to file
model_to_save = "rf"

with open("generated_model/body_language_model.pkl", "wb") as f:
    pickle.dump(fit_models[model_to_save], f)

## 4. Real-time Detections with Model <a id="real-time-detection"></a>

In [ ]:
with open("generated_model/body_language_model.pkl", "rb") as f:
    model_inference = pickle.load(f)

In [ ]:
# Start streaming video from webcam
video_stream()
# Label for video
label_html = "Real Time Detection"

drawing_lm = ''

with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

  while True:
      js_reply = video_frame(label_html, drawing_lm)
      
      if not js_reply:
          break
        
      # Convert JS response to OpenCV Image
      image = js_to_image(js_reply["img"])

      # Recolor feed
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

      # Make detections
      results = holistic.process(image)

      # Draw face landmarks
      mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS)

      # Right hand
      mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

      # Left hand
      mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

      # Pose 
      mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS)
      
      # Export coordinates
      if results.pose_landmarks and results.face_landmarks:
          # Extract pose landmarks
          pose = results.pose_landmarks.landmark
          pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                    for landmark in pose]).flatten())

              
          # Extract face landmarks
          face = results.face_landmarks.landmark
          face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                    for landmark in face]).flatten())
          
          # Concate row
          row = pose_row + face_row 
          
          # Predict using inference model
          X = pd.DataFrame([row])
          pred = model_inference.predict(X)[0]
          prob = np.max(model_inference.predict_proba(X)[0]).round(2)
          print(pred, prob)
          
          # Display result
          cv2.rectangle(image, (0,0), (250,60), (245, 117, 16), -1)
          cv2.putText(image, "CLASS", (95,12), cv2.FONT_HERSHEY_SIMPLEX,
                      0.5, (0,0,0), 1, cv2.LINE_AA)
          cv2.putText(image, pred, (90,40), cv2.FONT_HERSHEY_SIMPLEX,
                      1, (255, 255, 255), 2, cv2.LINE_AA)
          cv2.putText(image, "PROB", (15,12), cv2.FONT_HERSHEY_SIMPLEX,
                      0.5, (0,0,0), 1, cv2.LINE_AA)
          cv2.putText(image, str(prob), (10,40), cv2.FONT_HERSHEY_SIMPLEX,
                      1, (255, 255, 255), 2, cv2.LINE_AA)

      
      # Convert overlay of landmarks into bytes
      lm_bytes = drawing_array_to_bytes(image)
      
      # Update landmarks so next frame gets new overlay
      drawing_lm = lm_bytes

<IPython.core.display.Javascript object>

raise 0.81
raise 0.85
raise 0.85
raise 0.87
raise 0.86
raise 0.88
raise 0.93
raise 0.93
raise 0.9
raise 0.93
raise 0.88
raise 0.78
raise 0.71
raise 0.71
raise 0.78
raise 0.77
raise 0.69
raise 0.57
raise 0.64
raise 0.7
raise 0.68
raise 0.44
raise 0.45
raise 0.77
raise 0.6
raise 0.67
raise 0.67
raise 0.85
raise 0.78
raise 0.84
raise 0.81
raise 0.81
raise 0.72
raise 0.76
raise 0.76
raise 0.81
raise 0.81
raise 0.76
